In [ ]:
import pprint
import json
import pathlib
import pandas as pd
import os
import importlib
import networkx as nx
import matplotlib.pyplot as plt
import math

In [ ]:
from libWill.helpers import *
# importlib.reload(helpers)
pp = pprint.PrettyPrinter(indent=4)

In [ ]:
dirty_hourly = pathlib.Path('../../data/dirty/hourly/')
jsonl_files = os.listdir(dirty_hourly)

In [ ]:
all_tweets = []

for file in jsonl_files:
    all_tweets.extend(filter_tweet_fields_as_dict(pathlib.Path(dirty_hourly,file)))

In [ ]:
df = pd.DataFrame(all_tweets[0:5000])
df.tail()

In [ ]:
df = df[['created_at','screen_name','full_text','id','geo','lang','mention_ids','mentions','retweet_count','rt_status_id','rt_status_screen_name']]

In [ ]:
df.head(10)

In [ ]:
df_network = df[['screen_name','mentions','rt_status_screen_name']]
df_network.head()

In [ ]:
smallnet = df_network
smallnet


##Unpacking the mentions column to have multiple rows for each item in the mentions list

In [ ]:
smallnet_exploded = smallnet.explode('mentions')

In [ ]:
smallnet_exploded.reset_index(inplace=True,drop=True)

In [ ]:
smallnet_exploded.tail()

In [ ]:
edges = list(zip(smallnet_exploded['screen_name'],smallnet_exploded['mentions']))

In [ ]:
edges = [t for t in edges if not any(isinstance(n, float) and math.isnan(n) for n in t)]

In [ ]:
#print(edges)

In [ ]:
twit_graph = nx.Graph()

In [ ]:
twit_graph.add_nodes_from(smallnet['screen_name'])

In [ ]:
twit_graph.add_edges_from(edges)

In [ ]:
list(twit_graph.nodes)

In [ ]:
nx.draw(twit_graph)

In [ ]:
nx.density(twit_graph)

In [ ]:
nx.number_of_nodes(twit_graph)

In [ ]:
nx.number_of_edges(twit_graph)

In [ ]:
from bokeh.io import show, output_file
from bokeh.models import Plot, Range1d, MultiLine, Circle, HoverTool, BoxZoomTool, ResetTool
from bokeh.models.graphs import from_networkx
from bokeh.palettes import Spectral4
from bokeh.plotting import figure

In [101]:


plot = figure(title="5000 Tweets Test", x_range=(-1.1,1.1), y_range=(-1.1,1.1))
plot.add_tools(BoxZoomTool(), ResetTool())
graph = from_networkx(twit_graph, nx.spring_layout, scale=2, center=(0,0))
plot.renderers.append(graph)

output_file("5000_tweet_test2.html")
show(plot)